# How to define a graph schema with GQLAlchemy? 

Through this short tutorial, you will learn how to install Memgraph, connect to it from a Jupyter Notebook and use the GQLAlchemy library to create nodes and relationships in Memgraph.

## 1. Prerequisites

For this tutorial, you will need to install:
- [Jupyter](https://jupyter.org/install)
- [Docker](https://docs.docker.com/get-docker/)

Docker is used because Memgraph is a native Linux application and cannot be installed on Windows and macOS.

## 2. Installation using Docker

After you install Docker, you can set up Memgraph by running:

```
docker run -it -p 7687:7687 -p 3000:3000 memgraph/memgraph-platform
```

This command will start the download and after it finishes, run the Memgraph container.

## 3. Connecting to Memgraph with GQLAlchemy

We will be using the **GQLAlchemy** object graph mapper (OGM) to connect to Memgraph and execute **Cypher** queries easily. GQLAlchemy also serves as a Python driver/client for Memgraph. You can install it using:

```
pip install gqlalchemy
```

> **Hint**: You may need to install [CMake](https://cmake.org/download/) before installing GQLAlchemy.

Maybe you got confused when I mentioned Cypher. You can think of Cypher as SQL for graph databases. It contains many of the same language constructs like `CREATE`, `UPDATE`, `DELETE`... and it's used to query the database.

In [15]:
from gqlalchemy import Memgraph

In [16]:
memgraph = Memgraph("127.0.0.1", 7687)

Let's make sure that Memgraph is empty before we start with anything else.

In [17]:
memgraph.drop_database()

## 4. Creating nodes

First, we will need to create a class that will inherit from `gqlalchemy.Node`. Then, we can just create instances of the class and call their `save()` method to store them in the database:

In [18]:
from typing import Optional
from gqlalchemy import Node, Field


class User(Node):
    id: int = Field(index=True, unique=True, db=memgraph)
    name: Optional[str] = Field()
        
u1 = User(id=0, name='Ron')
u2 = User(id=1, name='Leslie')

u1.save(memgraph)
u2.save(memgraph)

<User id=1964 labels={'User'} properties={'id': 1, 'name': 'Leslie'}>

What’s going on here:

- `Node` is a Python class that maps to a graph object in `Memgraph`.
- Classes that inherit from `Node` map to a single label in the graph database.
- In this case, the class `User` maps to the label `:User`.
- The properties `id` and `name` are mapped to properties of the nodes labeled `User` in the graph database, their types must be defined and are enforced.
- If the type of the property is missing the `Optional` keyword, then it can’t be `None` or missing.
- `Field` is a function from pydantic that you can use to define constraints in the graph database like uniqueness constraints, indexes and existence constraints.
- Whenever you provide a constraint to the `Field` function, you have to provide a `Memgraph` object in the `db` argument as well, so those constraints can be enforced.
- In this instance, a uniqueness constraint and an index are added to Memgraph on the label `User` and the property `id` .
- `u1` is a `User` object that is created and saved to `Memgraph` in the following Cypher form: `(john1 :User {id: 1, name: "John"})`.

## 4. Creating relationships

First, we will need to create a class that will inherit from `gqlalchemy.Relationship`. Then, we can just create instances of the class and call their `save()` method to store them in the database:

In [20]:
from gqlalchemy import Relationship
from datetime import datetime


class Follows(Relationship, type='FOLLOWS'):
    pass

follows = Follows(
    _start_node_id=u1._id,
    _end_node_id=u2._id,
)
follows.save(memgraph)

<Follows id=83 start_node_id=1963 end_node_id=1964 nodes=(1963, 1964) type=FOLLOWS properties={}>

What’s going on here:

- `Relationship` is a Python class that maps to a graph object in `Memgraph`.
- Classes that inherit from `Relationship` map to a single type in the graph database.
- In this case, the class `Follows` maps to the type `:FOLLOWS`.
- The relationship has ne properties, but they can be added the same way as with the `User` class.
- When creating a relationship, the arguments `_start_node_id` and `_end_node_id` have to be specified.

## What's next?

Now it's time for you to use Memgraph on a graph problem!

You can always check out [Memgraph Playground](https://playground.memgraph.com/) for some cool use cases and examples.
If you need some help with GQLAlchemy, don't forget to check out the docs